In [1]:
import os
import numpy as np
import Model

from config import root, dir_dataset_train, dir_dataset_val, dir_resources

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_name = 'v2-resnet-50'
dir_model =  os.path.join(root, 'Model/', model_name)

# model = Model.create_model()
model = Model.load_model(dir_model)
Model.save_plot(os.path.join(dir_resources, 'model_plot', "%s-plot.png"%model_name), model)

Successfully created new model.
Successfully saved model plot to ../Resources/model_plot\v2-resnet-50-plot.png


In [3]:
def create_index(source):
    return (source, os.listdir(source))

def generator(source, index, num_batch, data_augmentation=True):
    while(True):
        np.random.shuffle(index)
        for i in range(num_batch):
            X, y = np.load( os.path.join(source, index[i]), allow_pickle=True )
            X = np.asarray( list(X), 'float32' )
            if(data_augmentation):
                X = Model.data_augmentation(X)
            y = np.asarray( list(y), 'float32' )
            yield(X, y)    

def create_generator(index, data_augmentation=True):
    source = index[0]
    index = index[1]
    num_batch = len(index)
    gen = generator(source, index, num_batch, data_augmentation)
    return gen, num_batch

In [4]:
train_index = create_index(dir_dataset_train)
val_index = create_index(dir_dataset_val)

train_gen, train_num_batch = create_generator(train_index)
val_gen, val_num_batch = create_generator(val_index, data_augmentation=False)

In [6]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath=dir_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

history = model.fit_generator(
    generator = train_gen,
    steps_per_epoch = train_num_batch,
    epochs = 35,
    validation_data = val_gen,
    validation_steps = val_num_batch,
    max_queue_size = 20,
    verbose = 1,
    callbacks = callbacks
)

Epoch 1/35
898/898 [==============================] - 429s 478ms/step - loss: 1.7813 - accuracy: 0.3992 - val_loss: 1.3924 - val_accuracy: 0.4954

Epoch 00001: val_loss improved from inf to 1.39238, saving model to ../Model\v2-resnet-50
INFO:tensorflow:Assets written to: ../Model\v2-resnet-50\assets
Epoch 2/35
898/898 [==============================] - 413s 460ms/step - loss: 1.5162 - accuracy: 0.4442 - val_loss: 1.4251 - val_accuracy: 0.4862

Epoch 00002: val_loss did not improve from 1.39238
Epoch 3/35
898/898 [==============================] - 404s 450ms/step - loss: 1.4168 - accuracy: 0.4693 - val_loss: 1.3064 - val_accuracy: 0.5113

Epoch 00003: val_loss improved from 1.39238 to 1.30644, saving model to ../Model\v2-resnet-50
INFO:tensorflow:Assets written to: ../Model\v2-resnet-50\assets
Epoch 4/35
898/898 [==============================] - 423s 471ms/step - loss: 1.3564 - accuracy: 0.4879 - val_loss: 1.2968 - val_accuracy: 0.5224

Epoch 00004: val_loss improved from 1.30644 to 1.

898/898 [==============================] - 404s 450ms/step - loss: 0.9945 - accuracy: 0.6274 - val_loss: 1.1753 - val_accuracy: 0.5887

Epoch 00034: val_loss did not improve from 1.14183
Epoch 35/35
898/898 [==============================] - 395s 440ms/step - loss: 0.9887 - accuracy: 0.6286 - val_loss: 1.1659 - val_accuracy: 0.5847

Epoch 00035: val_loss did not improve from 1.14183


In [8]:
model = Model.load_model(dir_model)
result = model.evaluate_generator(
    generator = val_gen,
    steps = val_num_batch,
    max_queue_size = 20,
    verbose = 1,
)
print('loss: %.4f'%result[0])
print('acc : %.2f%%'%(result[1]*100))

Successfully loaded model from ../Model/v2-resnet-50
225/225 [==============================] - 68s 297ms/step - loss: 1.1461 - accuracy: 0.5851
loss: 1.1461
acc : 58.51%
